# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
import re
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection  import GridSearchCV


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\omadz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\omadz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\omadz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disasterETLpipeline.db')
df = pd.read_sql_table('disasterETLpipeline', engine)
X = df.message
Y = df.loc[:,"related":"direct_report"]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9]', '', text)
    words = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens = [w for w in words if w not in stopwords.words('english')]
    cleanTokens = []
    for token in tokens:
        cleanToken = lemmatizer.lemmatize(token).lower().strip()
        cleanTokens.append(cleanToken)
    return cleanTokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-2)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002113C3D8C10>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-2)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# print('Training set score:', pipeline.score(X_train, Y_train) * 100, '%')
# print('Testing set score:', pipeline.score(X_test, Y_test) * 100, '%')

y_pred = pipeline.predict(X_test)
y_pred_df = pd.DataFrame(y_pred)
for i,col in enumerate(Y.columns):
        
        x = (col, '\n', classification_report(np.hstack(Y_test.values.astype(int)), np.hstack(y_pred_df.values.astype(int))), "***\n")
        # print(classification_report(np.hstack(Y_test.values.astype(int)), np.hstack(y_pred_df.values.astype(int))))
        # print("***\n")


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
parameters = {
    'clf__estimator__min_samples_leaf': [1,10],
    'clf__estimator__max_features': ['auto','log2'],
    'vect__smooth_idf':[True]
}

# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, Y_train)
y_pred_cv = cv.predict(X_test)
for i,col in enumerate(Y.columns):
        print(col)
        print(classification_report(np.hstack(Y_test.values.astype(int)), np.hstack(y_pred_cv.values.astype(int))))
        print("***\n")

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
])

parameters = {
    'vect__smooth_idf': [True,False],
}
    # train classifier
cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)
cv.fit(X_train, Y_train)
y_pred_ad = cv.predict(X_test)  

    # display results
for i,col in enumerate(Y.columns):
        print(col)
        print(classification_report(np.hstack(Y_test.values.astype(int)), np.hstack(y_pred_ad.values.astype(int))))
        print("***\n")

### 9. Export your model as a pickle file

In [20]:
with open('classifier.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.